In [1]:
import pandas as pd
import numpy as np
import optuna
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import tensorflow as tf
import ta
from fucnt_opt import TradingStrategyOptimizer
from fucnt_opt import optimize
from fucnt_opt import clean_ds
from fucnt_opt import dnn_strategy




/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-04-04 19:36:11.849961: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-04 19:36:11.850245: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 19:36:11.852649: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-04 19:36:11.877504: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instru

In [2]:
optimizer = TradingStrategyOptimizer('/home/rodo/code/proyecto_4/004_DL/data/close_data_buy_5.csv', '/home/rodo/code/proyecto_4/004_DL/data/close_data_sell_5.csv')
df = optimizer.run()
df

[I 2024-04-04 19:36:12,992] A new study created in memory with name: no-name-4e236584-8386-4957-8881-936e68b92e7a
/home/rodo/code/proyecto_4/004_DL/deep_learning/fucnt_opt.py:28: FutureWarning: suggest_loguniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use suggest_float(..., log=True) instead.
  lr = trial.suggest_loguniform('lr', 1e-4, 1e-2)
/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/keras/src/layers/core/dense.py:88: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
2024-04-04 19:36:13.599052: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:984] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been b

1236/1236 ━━━━━━━━━━━━━━━━━━━━ 1s 561us/step
1236/1236 ━━━━━━━━━━━━━━━━━━━━ 1s 498us/step


,Y_BUY_PRED,Y_SELL_PRED
0,True,False
1,True,False
2,True,False
3,True,False
4,True,False
...,...,...
39533,True,False
39534,True,False
39535,True,False
39536,True,False


In [3]:


df1 = pd.read_csv("../data/aapl_5m_train.csv")
df_5min = clean_ds(df1)
close_data = df_5min[['Timestamp','Close', 'X_t-1', 'X_t-2', 'X_t-3', 'X_t-4' ,'X_t-5','RSI', 'Y_BUY']]
close_data = close_data.dropna()

df_indexed = df.reset_index()
close_data_indexed = close_data.reset_index()

close_data_updated = close_data_indexed.join(df_indexed[['Y_BUY_PRED', 'Y_SELL_PRED']])



closes_5min = close_data_updated[['Timestamp', 'Close','Y_BUY_PRED', 'Y_SELL_PRED']]
closes_5min

,Timestamp,Close,Y_BUY_PRED,Y_SELL_PRED
0,1609778700,129.413299,True,False
1,1609779000,129.148498,True,False
2,1609779300,128.820007,True,False
3,1609779600,128.580001,True,False
4,1609779900,128.351699,True,False
...,...,...,...,...
39533,1672432800,129.054992,True,False
39534,1672433100,129.380004,True,False
39535,1672433400,129.764404,True,False
39536,1672433700,129.949996,True,False


In [4]:



cash = 1_000_000
active_operations = []
com = 0.00125  # comision en GBM
strategy_value = [1_000_000]
best_global_strategy = {'name': None, 'value': float('-inf')}

def optimize(trial):
    # Definición de los parámetros a optimizar
    stop_loss = trial.suggest_float('stop_loss', 0.00250, 0.05)
    take_profit = trial.suggest_float('take_profit', 0.00250, 0.05)
    n_shares = trial.suggest_int('n_shares', 5, 200)
    
    dnn_strat = dnn_strategy(
        df=closes_5min,  # df
        cash=cash,  # saldo inicial
        active_operations=[],
        com=com,  # comisión GBM
        n_shares=n_shares,
        stop_loss=stop_loss,
        take_profit=take_profit
    )

    dnn_strat.run_strategy()
    
    strategy_values = {
        'dnn_strategy': dnn_strat.run_strategy()
    }
    
    best_strategy_name = max(strategy_values, key=strategy_values.get)
    best_strategy_value = strategy_values[best_strategy_name]

    if best_strategy_value > best_global_strategy['value']:
        best_global_strategy['name'] = best_strategy_name
        best_global_strategy['value'] = best_strategy_value

    
    # Retorna el valor de la mejor estrategia
    return best_strategy_value

In [5]:
# Inicializar y ejecutar la optimización


study = optuna.create_study(direction='maximize')
study.optimize(optimize ,n_trials=1000,n_jobs=-1)

# Los mejores parámetros encontrados en el mejor trial
best_params = study.best_trial.params
best_value = study.best_trial.value

# Comparar con el mejor valor global previamente encontrado y el nombre de la estrategia
best_strategy_name = best_global_strategy['name']
best_strategy_value = best_global_strategy['value']

# Imprimir los resultados, incluido el nombre de la mejor estrategia global y su valor
print(f"Best buy overall strategy: {best_strategy_name} with value: {best_strategy_value}")
print("Best buy strategy parameters:", best_params)

[I 2024-04-04 19:39:19,050] A new study created in memory with name: no-name-9328d5b4-09a2-4296-8fc9-491ba716bb90
[W 2024-04-04 19:39:19,057] Trial 0 failed with parameters: {'stop_loss': 0.047299500240531776, 'take_profit': 0.02151483543458511, 'n_shares': 78} because of the following error: TypeError('Operation.__init__() takes 7 positional arguments but 8 were given').
Traceback (most recent call last):
  File "/home/rodo/code/proyecto_4/004_DL/.venv/lib/python3.11/site-packages/optuna/study/_optimize.py", line 200, in _run_trial
    value_or_values = func(trial)
                      ^^^^^^^^^^^
  File "/tmp/ipykernel_57929/220617117.py", line 23, in optimize
    dnn_strat.run_strategy()
  File "/home/rodo/code/proyecto_4/004_DL/deep_learning/fucnt_opt.py", line 183, in run_strategy
    self.active_operations.append(Operation(self.df,'Long', row.Close, row.Timestamp, n_shares, stop_loss, take_profit))
                                  ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

TypeError: Operation.__init__() takes 7 positional arguments but 8 were given